In [27]:
import pandas as pd
import seaborn as sns

In [28]:
df=pd.read_csv('home_appliances.csv',encoding= 'unicode_escape', sep=',')

In [29]:
df.head()

,Electricity(units),Age,Maintenance,Appliance type,CO2 Emissions
0,345,2,Regular,Washing Machine,0.6
1,450,3,Regular,Dryer,0.8
2,275,1,Irregular,Dishwasher,0.5
3,420,2,Regular,Refrigerator,0.7
4,200,5,Regular,Television,0.4


In [30]:
df.isna().sum()

Electricity(units)    0
Age                   0
Maintenance           0
Appliance type        0
CO2 Emissions         0
dtype: int64

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Electricity(units)  217 non-null    object
 1   Age                 217 non-null    object
 2   Maintenance         217 non-null    object
 3   Appliance type      217 non-null    object
 4   CO2 Emissions       217 non-null    object
dtypes: object(5)
memory usage: 8.6+ KB


In [31]:
corr=df.corr()
sns.heatmap(corr,annot=True,fmt='.1f')

/var/folders/mn/r97rfrgx549chn4x7dwcz_q80000gn/T/ipykernel_48992/1999486822.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr=df.corr()


ValueError: zero-size array to reduction operation fmin which has no identity

In [6]:
from sklearn.preprocessing import LabelEncoder

label_encoder1=LabelEncoder()
df['Maintenance']=label_encoder1.fit_transform(df['Maintenance'])

label_encoder2=LabelEncoder()
df['Appliance type']=label_encoder2.fit_transform(df['Appliance type'])

# label_encoder3=LabelEncoder()
# df['Fuel Type']=label_encoder3.fit_transform(df['Fuel Type'])

In [7]:
df.head()

,Electricity(units),Age,Maintenance,Appliance type,CO2 Emissions
0,345,2,1,23,0.6
1,450,3,1,6,0.8
2,275,1,0,5,0.5
3,420,2,1,17,0.7
4,200,5,1,19,0.4


In [8]:
le_name_mapping1 = dict(zip(label_encoder1.classes_, label_encoder1.transform(label_encoder1.classes_)))
print(le_name_mapping1)

{'Irregular': 0, 'Regular': 1}


In [9]:
le_name_mapping2 = dict(zip(label_encoder2.classes_, label_encoder2.transform(label_encoder2.classes_)))
print(le_name_mapping2)

{'Air Conditioner': 0, 'Blender': 1, 'Ceiling Fan': 2, 'Coffee Maker': 3, 'Computer': 4, 'Dishwasher': 5, 'Dryer': 6, 'Food Processor': 7, 'Freezer': 8, 'Furnace': 9, 'Hair Dryer': 10, 'Heater': 11, 'Ice Maker': 12, 'Iron': 13, 'Light Bulb': 14, 'Microwave': 15, 'Oven': 16, 'Refrigerator': 17, 'Stove': 18, 'Television': 19, 'Toaster': 20, 'Toaster Oven': 21, 'Vacuum Cleaner': 22, 'Washing Machine': 23, 'Water Heater': 24}


In [10]:
import pickle
with open('maintenance.pickle', 'wb') as handle:
    pickle.dump(le_name_mapping1, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('maintenance.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [11]:
import pickle
with open('appliance_type.pickle', 'wb') as handle:
    pickle.dump(le_name_mapping2, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('appliance_type.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [12]:
from sklearn.model_selection import train_test_split

X=df.drop('CO2 Emissions',axis=1)
y=df['CO2 Emissions']
X_train, X_test,y_train,y_test=train_test_split(X,y, test_size=0.2)

In [13]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()
rf.fit(X_train,y_train)

RandomForestRegressor()

In [14]:
rf.score(X_test,y_test)

0.9121754310344832

In [15]:
pickle.dump(rf, open('home_appliances_co2.pkl', 'wb'))

In [33]:
def predFunc2(electricity, age, maintenance, appliance_type):
    with open('maintenance.pickle', 'rb') as handle:
        b1 = pickle.load(handle)
    with open('appliance_type.pickle', 'rb') as handle:
        b2 = pickle.load(handle)
    model=pickle.load(open('home_appliances_co2.pkl', 'rb'))
    val1=b1.get(maintenance)
    val2=b2.get(appliance_type)
#     val3=b3.get(fuel_type)
    ans=model.predict([[electricity, age, val1, val2]])
    return ans[0]

In [34]:
predFunc2(345, 2, 'Regular', 'Washing Machine')

/Users/adwaitlaxmikantbhosale/miniforge3/envs/tf2/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


0.600000000000001

In [35]:
y_pred=rf.predict(X_test)

ValueError: could not convert string to float: 'Electricity(units)'

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import math

mse=mean_squared_error(y_test, y_pred)
rmse=math.sqrt(mse)
print(f"MAE: {mean_absolute_error(y_test, y_pred)}")
print(f"MAE: {mean_squared_error(y_test, y_pred)}")
print(f"RMSE: {rmse}")